# Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются.

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена.
Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:

In [2]:
path_to_AnnaKarenina = "data/AnnaKarenina.txt"
path_to_WarAndPeace = "data/WarAndPeace.txt"
path_to_WarAndPeaceEng = "data/WarAndPeaceEng.txt"

## Часть 1

1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
    * подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
    * возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
    * расшифруйте их таким частотным методом.


In [49]:
import os
import re
import random
import string
import numpy as np
from collections import Counter, defaultdict
from copy import copy
from tqdm import tqdm
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


In [40]:
#support function

def read_data(path):
    with open(path, "r", encoding="utf8") as file:
        data = file.read().replace('\n', ' ')
        data = re.sub('\W+',' ', data).lower()
    return data

def show_first_elem(d, i):
    cnt = 0
    for k, v in d.items():
        cnt+=1
        print(f'\"{k}\" :  {v}')
        if cnt == i:
            break

In [16]:
AnnaKarenina = read_data(path_to_AnnaKarenina)
WarAndPeace = read_data(path_to_WarAndPeace)
WarAndPeaceEng = read_data(path_to_WarAndPeaceEng)

In [14]:
AnnaKarenina[:150]

'Annotation   «Анна Каренина», один из самых знаменитых романов Льва Толстого, начинается ставшей афоризмом фразой: «Все счастливые семьи похожи друг н'

In [17]:
AnnaKarenina[:150]

'annotation анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга '

In [77]:
codes = np.random.choice(np.arange(10000), size=15, replace=False)
chars_in_text = list(set(AnnaKarenina))
symbols = list(map(chr, codes))

In [78]:
symbols

['ᶷ',
 '൫',
 'Ჽ',
 'Შ',
 'ᅡ',
 'Ꭲ',
 'ߎ',
 'ڣ',
 'ᛁ',
 'ᮊ',
 'Ⴋ',
 'ᮦ',
 '\u1ac4',
 'ᵷ',
 'ᑃ']

In [75]:
codes

array([2228, 8815, 1831, 5721,  738, 6729, 8373, 8596, 2678, 5842, 4150,
       1756, 4590, 3350, 7502])

In [ ]:
def text_encoder(text):
    chars_in_text = list(set(text))
    cipher = string.printable
    random.shuffle(cipher)
    
    pass

In [71]:
a = list(set(string.printable))
random.shuffle(a)
a

['I',
 'f',
 'w',
 '6',
 ')',
 '0',
 '_',
 'C',
 'B',
 '5',
 '$',
 'X',
 'M',
 '\\',
 'p',
 '-',
 'Q',
 '{',
 ',',
 ' ',
 'n',
 '=',
 '\t',
 '&',
 'Y',
 'o',
 'm',
 'r',
 '8',
 'W',
 'L',
 's',
 'z',
 '|',
 'F',
 'R',
 '2',
 '#',
 'i',
 '*',
 'E',
 '<',
 'U',
 'J',
 'v',
 'g',
 'h',
 '?',
 'k',
 '\x0b',
 '4',
 'K',
 'O',
 'P',
 "'",
 '%',
 '\r',
 'S',
 'd',
 'e',
 '/',
 ':',
 '`',
 '"',
 'A',
 'b',
 't',
 '\n',
 '9',
 '.',
 ';',
 'H',
 ']',
 'G',
 'j',
 '~',
 'N',
 '>',
 '3',
 'q',
 'D',
 '!',
 'l',
 'a',
 '\x0c',
 '^',
 '+',
 '}',
 'u',
 '[',
 '1',
 'x',
 '(',
 '@',
 'y',
 'T',
 'c',
 'V',
 'Z',
 '7']

In [67]:
class HolmesCipher:
    def __init__(self, data):
        self.data = data
    
    def frequency_mapping(self):
        char_cnt = Counter(self.data)
        sorted_char_cnt = sorted(char_cnt.items(), key=lambda item: item[1], reverse=True )
        char_cnt = {key: value/len(self.data) for key, value in sorted_char_cnt} 
        
        return char_cnt
    
    def cipher_map(self):
        
    
    
    
    def encode(self):
        pass
    
    def decode(self):
        pass
    
    
    

IndentationError: expected an indented block (1520554673.py, line 17)

In [47]:
cipher_1 = HolmesCipher(AnnaKarenina)

anna_k_freq_dict = cipher_1.frequency_map

show_first_elem(anna_k_freq_dict, 8)

NameError: name '_frequency_mapping' is not defined

In [ ]:
def get_unigramm_mapping(encoded, train):
    char_cnt = Counter(train)
    encoded_cnt = Counter(encoded).most_common()
    reverse_mapping = {}
    for i, (char, _) in enumerate(char_cnt.most_common(len(encoded_cnt))):
        reverse_mapping[encoded_cnt[i][0]] = char
    return reverse_mapping


    def get_sorted_frequency_dict(self):
        """
        input: text
        output: sorted dict freq symbol
        """
        counts = dict(Counter(self.text)) 
        counts = {k: v/len(self.text) for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True )}        
        return counts

    def get_encode_dict(self):
        freq_dict = self.get_sorted_frequency_dict()
        shuffled_letters = list(freq_dict.keys())
        random.shuffle(shuffled_letters)
        return {list(freq_dict.keys())[i]:shuffled_letters[i] for i in range(len(shuffled_letters))}

    def get_decode_dict(self, freq_letters_list):
        freq_dict = self.get_sorted_frequency_dict()
        decode_dict = {list(freq_dict.keys())[i]:freq_letters_list[i] for i in range(min(len(freq_dict),len(freq_letters_list)))}
        return decode_dict

    def encode(self):
        encode_dict = self.get_encode_dict()
        return ''.join([encode_dict[i] for i in self.text])

    def decode(self, freq_dict):
        decode_dict = self.get_decode_dict(freq_dict)
        return ''.join([decode_dict[i] if i in decode_dict.keys() else '?' for i in self.text])
